## MMOE

```
实现细节：
    * 不考虑embedding layer的模型训练
    * FakeMultiTaskDataFactory直接合成concatenation后的数据
    * 共四个输出头-TaskHead（点击率，点赞率，收藏率，转发率）
    * 每个输出头对应一个TaskGate，用来merge专家输出。
    
    * Expert 多目标专家模型可以替换成任意fea-cross模型，
        专家数量是超参数，这里默认使用MLP。
    
    * TaskGate中，Softmax输出的𝑛 个数值被mask 的概率都是10%。
•       每个“专家”被随机丢弃的概率都是10%。
```

In [1]:
%cd /playground/sgd_deep_learning/sgd_rec_sys/
import sys 
sys.path.append('./python')

/playground/sgd_deep_learning/sgd_rec_sys


In [2]:
import torch
from torch import nn
from torch.utils.data import DataLoader
import torch.optim as optim

import pickle
import numpy as np

from sgd_rec_sys.data import FakeMultiTaskDataFactory, MultiTaskDataset
from sgd_rec_sys.rank import MMOE, Expert, TaskHead, TaskGate, CrossEntropyLoss


In [3]:
## 数据准备 ##

# input
n_samples = 500 # 总样本数
fea_emb_dim = 200
target_num = 4

## 生成伪CTR数据
dump_file = './data/fake/tmp_dssm.pkl'
fake_data_factory = FakeMultiTaskDataFactory(n_samples,
                                        fea_emb_dim,
                                        target_num,
                                        dtype=np.float32)
fake_data_factory.presist(dump_file)

feature embedding success, shape: (500, 200)
targets success, shape: (500, 4)


In [4]:
## 参数设置 ##
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print('device:', device)

# 训练参数
train_batch_size = 64
epochs = 5

with open(dump_file,'rb') as f:
    fake_data = pickle.load(f)
train_ds = MultiTaskDataset(data_info=fake_data, device=device)
train_dl = DataLoader(train_ds, batch_size=train_batch_size, shuffle=True)

# 网络结构

expert_hidden_dims = [1024,256,32] # 包括最后一层
gate_hidden_dims = [256] # 不包括最后的输出层 （last_layer+softmax ->p1,p2,p3...)

head_in_dim = expert_hidden_dims[-1] # head的输入是baseNN的输出维度
head_hidden_dims = [] # []空隐藏层, 输出头默认一层映射 in_dim -> 1 -> sigmoid 

expert_num = 3 # 三个专家
task_num = target_num # 任务数由需要预测的target label决定

# 定义模型
expert_model_list = nn.ModuleList([Expert(in_dim=fea_emb_dim,
                                            hidden_dims = expert_hidden_dims,
                                            activation_fun=nn.ReLU())
                                    for _ in range(expert_num)])

# 每个task分配一个对立的gate control
gate_list = nn.ModuleList([TaskGate(in_dim=fea_emb_dim, 
                                         hidden_dims=gate_hidden_dims, 
                                         expert_num= expert_num, 
                                         activation_fun=nn.ReLU(), 
                                         dropout_p=0.1) # 10%专家丢弃率防止极化
                                for _ in range(task_num)])

# 一共四个输出头，new OutputHead 共生成四个对象，不共享
head_list = nn.ModuleList([TaskHead(in_dim=head_in_dim,
                                        hidden_dims = head_hidden_dims,
                                        activation_fun=nn.ReLU())
                                for _ in range(task_num)])

model = MMOE(expert_nets=expert_model_list, 
             heads_nets=head_list,
             gate_nets=gate_list).to(device)

print(model)

criterion = CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.01)  # SGD 优化器
# optimizer = torch.optim.Adam(model.parameters(), lr=1e-1, weight_decay=0.3)

device: cuda
MMOE(
  (experts): ModuleList(
    (0-2): 3 x Sequential(
      (0): Linear(in_features=200, out_features=1024, bias=True)
      (1): ReLU()
      (2): Linear(in_features=1024, out_features=256, bias=True)
      (3): ReLU()
      (4): Linear(in_features=256, out_features=32, bias=True)
      (5): ReLU()
    )
  )
  (heads): ModuleList(
    (0-3): 4 x Sequential(
      (0): Linear(in_features=32, out_features=1, bias=True)
      (1): Softmax(dim=None)
    )
  )
  (gates): ModuleList(
    (0-3): 4 x Sequential(
      (0): Linear(in_features=200, out_features=256, bias=True)
      (1): ReLU()
      (2): Linear(in_features=256, out_features=3, bias=True)
      (3): Softmax(dim=None)
      (4): Dropout(p=0.1, inplace=False)
    )
  )
)


/opt/conda/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
## 训练 ##
def train(dataloader, model, epochs=1,):
    for epoch in range(epochs):
        for x in dataloader:
            fea_embs, targets = x
            # 前向传播
            output = model(fea_embs)
            loss = criterion(output, targets)
        
        # 反向传播和优化
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        # log
        print(f'Epoch [{epoch+1}/{epochs}], Loss: {loss.item():.4f}')
            
train(train_dl, model, epochs)

/opt/conda/lib/python3.10/site-packages/torch/nn/modules/module.py:1518: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  return self._call_impl(*args, **kwargs)


Epoch [1/5], Loss: 211.5385
Epoch [2/5], Loss: 180.7692
Epoch [3/5], Loss: 178.8462
Epoch [4/5], Loss: 180.7692
Epoch [5/5], Loss: 215.3846
